## Laden und durchsuchen von eigenen Daten

1. Zuerst laden wir die Dokumente aus dem docs Verzeichniss
2. Dann teilen wir die Dokumente in kleine Segmente auf
3. Dann erstellen wir daraus einen Index

In [3]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('./docs', glob="**/*.md")
data = loader.load()
len(data)

2

In [4]:
print(f"Zeichen in Text {len(data[1].page_content)}")
data[1]

Zeichen in Text 38


Document(metadata={'source': 'docs/index.md'}, page_content='Personen\n\nOliver Guhr, Schulungsleiter')

### Wie können wir die Texte in kürzere Abschnitte unterteilen?

Web Demo [Chunk Visualizer](https://huggingface.co/spaces/m-ric/chunk_visualizer)

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=50)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))
all_splits

107


[Document(metadata={'source': 'docs/mms.md'}, page_content='Telekom MMS'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='Die Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='Unternehmen bei der digitalen Transformation versteht.'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='Über 2.200 Mitarbeitende an neun Standorten in Deutschland entwickeln zukunftsfähige'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='in Deutschland entwickeln zukunftsfähige Geschäftsmodelle für digitale Erlebnisse. Telekom MMS'),
 Document(metadata={'source':

In [6]:
import chromadb
from chromadb.utils import embedding_functions

# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
chroma = chromadb.Client()

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="paraphrase-multilingual-MiniLM-L12-v2")
#chroma.delete_collection("documents")
collection = chroma.get_or_create_collection("documents",embedding_function=sentence_transformer_ef)

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=[item.page_content for item in all_splits], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[item.metadata for item in all_splits], # filter on these!
    ids=[str(id) for id in range(0,len(all_splits))], # unique for each doc
)

/home/ogu/source/workshop-grundlagen-ai/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/ogu/source/workshop-grundlagen-ai/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["Ehemalige Geschäftsführer"],
    n_results=4,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    #where_document={"$contains":"Oliver"}  # optional filter
)

results

{'ids': [['49', '46', '48', '59']],
 'distances': [[5.316214084625244,
   5.756248474121094,
   7.247669219970703,
   9.02979850769043]],
 'metadatas': [[{'source': 'docs/mms.md'},
   {'source': 'docs/mms.md'},
   {'source': 'docs/mms.md'},
   {'source': 'docs/mms.md'}]],
 'embeddings': None,
 'documents': [['kaufmännischer Leiter des Unternehmens. Ehemalige Geschäftsführer sind: Friedhelm Theis (1995–1998)',
   'Die Geschäftsführung besteht aus einer Doppelspitze: Ralf Pechmann, mit Hauptverantwortung im',
   'Vertrieb, Marketing, Kunden und Marcus Gaube als kaufmännischer Leiter des Unternehmens. Ehemalige',
   'Preisträger des Ludwig-Erhard-Preises in der Kategorie „Mittlere Unternehmen“, als auch des vom']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

## Aufgaben 

* Trage informationen ein - Findet sie die suche wieder?
* Wie groß sollte die Chuck Size sein?
* Finde die ehemaligen Geschäftsführer


Optional 
* Open AI Embeddings
* BGE Embeddings